In [1]:
from typing import Any, Dict, List

from routrie import Router as RoutrieRouter
from http_router import Router as HTTPRouter
from starlette.routing import Route, Router as StarletteRouter

In [2]:
routes: Dict[str, Any] = {}

async def endpoint(*args: Any) -> Any:
    ...

# From https://github.com/klen/py-frameworks-bench
for n in range(5):
    routes[f"/route-{n}"] = endpoint
    routes[f"/route-dyn-{n}/{{part}}"] = endpoint

In [3]:
paths_to_match: List[str] = []
for n in range(1_000):
    paths_to_match.append("/route-0")
    paths_to_match.append("/route-1")
    paths_to_match.append("/route-2")
    paths_to_match.append("/route-3")
    paths_to_match.append("/route-4")
    paths_to_match.append(f"/route-dyn-0/foo-{n}")
    paths_to_match.append(f"/route-dyn-1/foo-{n}")
    paths_to_match.append(f"/route-dyn-2/foo-{n}")
    paths_to_match.append(f"/route-dyn-3/foo-{n}")
    paths_to_match.append(f"/route-dyn-4/foo-{n}")

In [4]:
routrie_router = RoutrieRouter({path.replace("{part}", ":part"): val for path, val in routes.items()})

In [5]:
%%timeit
for path in paths_to_match:
    routrie_router.find(path)

3.23 ms ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
http_router = HTTPRouter()
for path, value in routes.items():
    http_router.route(path)(value)

In [7]:
%%timeit
for path in paths_to_match:
    http_router(path)

8.19 ms ± 46.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
starlette_router = StarletteRouter(
    routes=[
        Route(path, endpoint)
        for path, endpoint in routes.items()
    ]
)

scopes_to_match = [
    {
        "type": "http",
        "method": "GET",
        "path": path
    }
    for path in paths_to_match
]

In [9]:
%%timeit
# simulate what Starlette does internally
for scope in scopes_to_match:
    for route in starlette_router.routes:
        match, _ = route.matches(scope)
        if match == match.FULL:
            break

27 ms ± 332 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
